In [ ]:
!pip install onnxruntime


In [ ]:
input_size=11
output_size=4

In [ ]:
import torch
import torch.onnx
import os
from torch import nn

# Load the trained model
model = nn.Sequential(
    nn.Linear(input_size, 128),
    nn.ReLU(),
    nn.Dropout(0.05),  # Lower dropout
    nn.Linear(128, 64),
    nn.ReLU(),         # Consistent activation
    nn.Linear(64, 16),
    nn.ReLU(),
    nn.Dropout(0.05),
    nn.Linear(16, output_size)  # No Softmax here
)

# Load weights from the trained model
model.load_state_dict(torch.load('final_model.pt'))
model.eval()  # Set the model to evaluation mode

# Specify a dummy input with the correct shape (e.g., input_size features)
dummy_input = torch.randn(1, input_size)  # Batch size 1, with input_size features

# Path to save the ONNX model
onnx_model_path = 'final_model.onnx'

# Export the model
torch.onnx.export(model, dummy_input, onnx_model_path, export_params=True, opset_version=12,
                  do_constant_folding=True, input_names=['input'], output_names=['output'])

print(f"Model has been exported to {onnx_model_path}")


In [ ]:
import onnxruntime as ort
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, accuracy_score
import matplotlib.pyplot as plt

# Set the number of classes (output size)
output_size = 4  # Number of output classes (adjust this based on your model's output)

# Define the class labels (adjust based on your dataset)
class_labels = [f"Class {i}" for i in range(output_size)]

# Load the ONNX model
onnx_model_path = 'final_model.onnx'
session = ort.InferenceSession(onnx_model_path)

# Get the input and output names from the ONNX model
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print(f"Input name: {input_name}, Output name: {output_name}")

# Prepare the test data (assuming `X_test` and `y_test` are pandas DataFrames or numpy arrays)
X_test_tensor = X_test.values.astype(np.float32)  # Convert to numpy array if it's a pandas DataFrame
y_test_tensor = y_test.values  # Actual labels for the test set

# Initialize variables to store results
y_true = []
y_pred = []
y_probs = []  # To store probabilities for metrics like ROC-AUC

# Evaluate the model on the test set using ONNX Runtime
batch_size = 1 # You can adjust the batch size here
for i in range(0, len(X_test_tensor), batch_size):  # Iterate through batches
    batch_input = X_test_tensor[i:i+batch_size]
    batch_labels = y_test_tensor[i:i+batch_size]
    
    # Run inference on the batch
    outputs = session.run([output_name], {input_name: batch_input})
    
    # Get the predicted labels (class with max probability)
    predicted = np.argmax(outputs[0], axis=1)  # Get the class with the max probability
    probabilities = outputs[0]  # Raw probabilities for each class
    
    # Append results to lists
    y_true.extend(batch_labels)
    y_pred.extend(predicted)
    y_probs.extend(probabilities)

# Convert results to NumPy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_probs = np.array(y_probs)  # Probabilities for each class

# **1. Classification Report**
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# **2. Confusion Matrix**
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# **3. Accuracy**
accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# **4. Precision, Recall, and F1-Score**
precision = cm.diagonal() / cm.sum(axis=0)
recall = cm.diagonal() / cm.sum(axis=1)
f1_scores = 2 * (precision * recall) / (precision + recall)

print("\nPer-Class Metrics:")
for i, (p, r, f1) in enumerate(zip(precision, recall, f1_scores)):
    print(f"Class {i} - Precision: {p:.4f}, Recall: {r:.4f}, F1-Score: {f1:.4f}")

# **5. ROC-AUC (for multi-class)**
if output_size == 2:
    # For binary classification
    roc_auc = roc_auc_score(y_true, y_probs[:, 1])
    print(f"ROC-AUC Score: {roc_auc:.4f}")

    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_probs[:, 1])
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()
else:
    # For multi-class ROC-AUC (one-vs-rest approach)
    roc_auc_ovr = roc_auc_score(y_true, y_probs, multi_class='ovr')
    print(f"Multi-Class ROC-AUC (OVR): {roc_auc_ovr:.4f}")

# **6. Log Predictions (optional for debugging)**
np.savetxt("predictions.csv", np.column_stack((y_true, y_pred)), fmt='%d', delimiter=',', header='True,Predicted')
